# Trabalho 1 - MC853B

**Alunos:** 

Lucca Miranda Nunes - XXXX

Luiz Felipe Corradini Rego Costa - 230613

Pablo Henrique Almeida Mendes - XXXX




**Doença de notificação compulsória escolhida:** Esquistossomose

## 1) Upload e investigação dos dados

### 1.1) Análise Exploratória dos dados

Os dados escolhidos para treinamento foram os da doença esquistossomose dos anos de 2021, 2022 e 2023. Os anos de 2024 em diante não estavam disponibilizados na base do SINAN. Todos os dados estão no diretório source/csv.

In [489]:
import pandas as pd

esq_20 = pd.read_csv('source/csv/ESQUBR20.csv')
esq_21 = pd.read_csv('source/csv/ESQUBR21.csv')
esq_22 = pd.read_csv('source/csv/ESQUBR22.csv')
esq_23 = pd.read_csv('source/csv/ESQUBR23.csv')

In [490]:
print(esq_20.shape)
print(esq_21.shape)
print(esq_22.shape)
print(esq_23.shape)

(1873, 54)
(2395, 54)
(2640, 54)
(3059, 54)


In [491]:
esq_20['Ano_atual'] = 2020
esq_21['Ano_atual'] = 2021
esq_22['Ano_atual'] = 2022
esq_23['Ano_atual'] = 2023

Com os três anos selecionados, obtemos, em primeiro momento, um total de 8094 instâncias. 

Precisamos investigar a quantidade de atributos-alvo em cada ano, a fim de garantir balanceamento nas classes dentro dos datasets de treino e de teste:

Valores correspondentes a cada desfecho possível do paciente:


1 - Cura

2 - Não Cura

3 - Óbito por 
esquistossomose 

4 -  Óbito por outras 
causas 

9 - Ignorado


Como iremos usar os dados dos três anos, faz mais sentido concatená-los em um único DataFrame, a fim de realizar a análise em um único comando

In [492]:
esq_total = pd.concat([esq_20, esq_21, esq_22, esq_23])
esq_total['EVOLUCAO'].value_counts()

1.0    4901
9.0    1121
3.0     242
2.0     225
4.0     113
Name: EVOLUCAO, dtype: int64

In [493]:
print(f"Porcentagem de mortes por esquistossomose: {len(esq_total[esq_total['EVOLUCAO'] == 3]) / len(esq_total) * 100 :.2f}%")

Porcentagem de mortes por esquistossomose: 2.43%


### 1.2) Divisão do conjunto de treino e teste

O primeiro passo vai ser separar um conjunto de teste nos dados, já que esse conjunto não deve ser pré-processado como os de treinamento. Para garantir que 

In [494]:
# Avaliar a janela de tempo entre o primeiro atendimento e a morte
# Avaliar como vai ser a divisão dos dados, e avaliar também como garantir 10% de representatividade de cada classe nos conjuntos de treino e teste

## 2) Pré-processamento dos dados

Podemos ver que, pelo número elevado de colunas, será necessário realizar uma análise quanto às colunas necessárias, para posteriormente avaliar se o total de instâncias apresentadas nos três anos serão suficientes para treinar o modelo.

### 2.1) Removendo Colunas desnecessárias

O primeiro passo para pré-processar será **remover colunas desnecessárias do dataset**. 

Depois de uma primeira análise, a equipe selecionou 11 colunas que julgamos como mais relevantes para o treinamento do modelo:


*colocar info das colunas*


In [495]:
esq_total.columns

Index(['Unnamed: 0', 'TP_NOT', 'ID_AGRAVO', 'DT_NOTIFIC', 'SEM_NOT', 'NU_ANO',
       'SG_UF_NOT', 'ID_MUNICIP', 'ID_REGIONA', 'ID_UNIDADE', 'DT_SIN_PRI',
       'SEM_PRI', 'ANO_NASC', 'NU_IDADE_N', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA',
       'CS_ESCOL_N', 'SG_UF', 'ID_MN_RESI', 'ID_RG_RESI', 'ID_PAIS',
       'DT_INVEST', 'ID_OCUPA_N', 'DT_COPRO', 'AN_QUANT', 'OUTROS', 'TRATAM',
       'TRATANAO', 'STCURA1', 'STCURA2', 'STCURA3', 'FORMA', 'TPAUTOCTO',
       'COUFINF', 'COPAISINF', 'COMUNINF', 'NOPROPIN', 'NOCOLINF',
       'DOENCA_TRA', 'EVOLUCAO', 'DT_ENCERRA', 'DT_DIGITA', 'DT_TRANSUS',
       'DT_TRANSDM', 'DT_TRANSSM', 'DT_TRANSRS', 'DT_TRANSSE', 'DT_OBITO',
       'DS_FORMA', 'AN_QUALI', 'DTTRAT', 'DT_RESU3', 'OUTRO_EX', 'Ano_atual'],
      dtype='object')

In [531]:
esq_total['COUFINF'].value_counts()

31.0    4380
29.0    1198
26.0     448
28.0     299
27.0     283
25.0     238
33.0     209
35.0     130
21.0      69
23.0      66
15.0      52
24.0      51
41.0      39
11.0      28
32.0      16
51.0      16
52.0      15
17.0      11
53.0       8
50.0       7
22.0       7
43.0       5
42.0       4
16.0       3
13.0       3
12.0       1
Name: COUFINF, dtype: int64

In [496]:
# Lista das colunas conforme dicionário SINAN
colunas_selecionadas = [
    'CS_GESTANT',                     
    'AN_QUANT',
    'AN_QUALI',            
    'TRATAM', 
    'FORMA',      
    'CS_ESCOL_N', 
    'CS_SEXO',  
    'ANO_NASC',
    'COUFINF',
    'EVOLUCAO',
    'Ano_atual'
]

# Seleção das colunas
esq_filtrado = esq_total[colunas_selecionadas]

esq_filtrado.head()


,CS_GESTANT,AN_QUANT,AN_QUALI,TRATAM,FORMA,CS_ESCOL_N,CS_SEXO,ANO_NASC,COUFINF,EVOLUCAO,Ano_atual
0,6,0.0,3.0,3.0,4.0,9.0,M,1969.0,31.0,2.0,2020
1,6,1.0,1.0,1.0,NaN,NaN,M,1984.0,26.0,1.0,2020
2,5,0.0,NaN,1.0,NaN,1.0,F,1975.0,35.0,1.0,2020
3,5,0.0,1.0,3.0,NaN,1.0,F,1977.0,29.0,NaN,2020
4,5,0.0,1.0,1.0,1.0,9.0,F,2004.0,27.0,1.0,2020


Renomeando as colunas selecionadas para garantir melhor entendimento:

In [497]:
novos_nomes_doenças = {
    'CS_GESTANT': 'Gestante',
    'AN_QUANT': 'Quantidade ovos encontrados',
    'AN_QUALI': 'Exame_qualitativo',
    'TRATAM': 'Tratamento realizado',
    'FORMA': 'Forma clínica',
    'CS_ESCOL_N': 'Escolaridade',
    'CS_SEXO': 'Sexo',
    'ANO_NASC': 'Ano de nascimento',
    'COUFINF': 'Unidade Federativa',
    'EVOLUCAO': 'Resultado'

}

esq_filtrado = esq_filtrado.rename(columns=novos_nomes_doenças)

print(esq_filtrado.shape)
esq_filtrado.head()

(9967, 11)


,Gestante,Quantidade ovos encontrados,Exame_qualitativo,Tratamento realizado,Forma clínica,Escolaridade,Sexo,Ano de nascimento,Unidade Federativa,Resultado,Ano_atual
0,6,0.0,3.0,3.0,4.0,9.0,M,1969.0,31.0,2.0,2020
1,6,1.0,1.0,1.0,NaN,NaN,M,1984.0,26.0,1.0,2020
2,5,0.0,NaN,1.0,NaN,1.0,F,1975.0,35.0,1.0,2020
3,5,0.0,1.0,3.0,NaN,1.0,F,1977.0,29.0,NaN,2020
4,5,0.0,1.0,1.0,1.0,9.0,F,2004.0,27.0,1.0,2020


### 2.2) Selecionando os dados a serem usados

Como o problema que iremos prever é de natureza binária (óbito ou não-óbito), precisamos remover todas as instâncias que a resposta foi "Ignorado" ou "Não-Cura". Se o caso foi ignorado, não podemos inserí-lo em nenhuma das duas classes, e a classificação de "Não-Cura" também não torna possível avaliar se o paciente veio a óbito ou não. Além disso, avaliamos que fugia do escopo do projeto considerar as mortes por outros fatores, então também retiramos essa classe da consideração.

In [498]:
esq_valid = esq_filtrado[(esq_filtrado['Resultado'] == 1) | (esq_filtrado['Resultado'] == 3)]
esq_valid['Resultado'].value_counts()

1.0    4901
3.0     242
Name: Resultado, dtype: int64

In [499]:
print(f"Porcentagem de mortes por esquistossomose: {len(esq_valid[esq_valid['Resultado'] == 3]) / len(esq_valid) * 100 :.2f}%")

Porcentagem de mortes por esquistossomose: 4.71%


### 2.3) Tratamento de atributos categóricos

Primeiramente, vamos visualizar nossos dados a fim de transformar os atributos categóricos em numéricos:

In [500]:
esq_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5143 entries, 1 to 3058
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Gestante                     5143 non-null   int64  
 1   Quantidade ovos encontrados  5143 non-null   float64
 2   Exame_qualitativo            4053 non-null   float64
 3   Tratamento realizado         5143 non-null   float64
 4   Forma clínica                3709 non-null   float64
 5   Escolaridade                 4551 non-null   float64
 6   Sexo                         5143 non-null   object 
 7   Ano de nascimento            5112 non-null   float64
 8   Unidade Federativa           4102 non-null   float64
 9   Resultado                    5143 non-null   float64
 10  Ano_atual                    5143 non-null   int64  
dtypes: float64(8), int64(2), object(1)
memory usage: 482.2+ KB


Podemos notar que, apesar de que o tipo de muitos dos atributos esteja definido como 'float', de acordo com o dicionário da doença, cada número representa uma característica distinta. Por isso, a ideia será realizar o One-Hot encoding para separar tais atributos categóricos.

#### 2.3.1) Gestante

In [501]:
esq_valid['Gestante'].value_counts()

6    3573
5    1422
9     111
2      14
4      12
1       8
3       3
Name: Gestante, dtype: int64

In [502]:
# Para gestante, consideraremos apenas como uma variável binária. 
# 5, 6 e 9 indica que não é gestante, e todas as demais indicam que é gestante

esq_valid['nao_gestante'] = (esq_valid['Gestante'].isin([5, 6, 9])).astype(int)
esq_valid['gestante'] = (~esq_valid['Gestante'].isin([5, 6, 9])).astype(int)


/tmp/ipykernel_8265/433438737.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  esq_valid['nao_gestante'] = (esq_valid['Gestante'].isin([5, 6, 9])).astype(int)
/tmp/ipykernel_8265/433438737.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  esq_valid['gestante'] = (~esq_valid['Gestante'].isin([5, 6, 9])).astype(int)


In [503]:
esq_valid[esq_valid['nao_gestante'] == 0]['Resultado'].value_counts()

1.0    37
Name: Resultado, dtype: int64

Após análise do fator gestante, é possível notar que um número inexpressivo de pessoas na base de dados são gestantes, e nenhuma delas apresenta veio a óbito. Adicionar tal feature na base pode corroborar com o viés do modelo, por isso optamos por retirar a categoria da base de dados.

In [504]:
esq_valid = esq_valid.drop(columns=['Gestante', 'nao_gestante','gestante'])

#### 2.3.2) Quantidade de ovos de schistossoma encontrados

Para o atributo "Quantidade de Ovos encontrados", este já está em forma binária, indicando se não foram encontrados ovos da Schistossoma no exame de fezes (0) ou se foram encontrados (1)

In [505]:
esq_valid['Quantidade ovos encontrados'].value_counts()

0.0    2690
1.0    2453
Name: Quantidade ovos encontrados, dtype: int64

#### 2.3.3) Exame qualitativo

Já para o atributo "Resultado exame ovos qualitativo", que indica o resultado (1=Positivo, 2=Negativo e 3=Não realizado), como são apresentadas mais de duas categorias, será necessário transformar em mais colunas.

Além disso, como existem muitos valores nulos na coluna, vamos atribuí-los a uma quarta categoria: Desconhecido

In [506]:
print(esq_valid['Exame_qualitativo'].value_counts())
print("Valores nulos: ", esq_valid['Exame_qualitativo'].isna().sum())

1.0    2254
3.0    1397
2.0     402
Name: Exame_qualitativo, dtype: int64
Valores nulos:  1090


In [507]:
# Adicionando "Desconhecido"
esq_valid['Exame_qualitativo'].fillna("Desconhecido", inplace=True)

In [508]:
# Definindo valores numéricos como texto
esq_valid['Exame_qualitativo'].replace(1, "Positivo", inplace=True)
esq_valid['Exame_qualitativo'].replace(2, "Negativo", inplace=True)
esq_valid['Exame_qualitativo'].replace(3, "Nao_Realizado", inplace=True)

In [509]:
# Criando novas colunas com one-hot-encoding
df_onehot = pd.get_dummies(esq_valid['Exame_qualitativo'], prefix='Exame', dtype=int)

# Concatenar com o DataFrame original
esq_valid = pd.concat([esq_valid, df_onehot], axis=1)

# Remover a coluna original
esq_valid.drop(columns=['Exame_qualitativo'], inplace=True)

In [510]:
esq_valid

,Quantidade ovos encontrados,Tratamento realizado,Forma clínica,Escolaridade,Sexo,Ano de nascimento,Unidade Federativa,Resultado,Ano_atual,Exame_Desconhecido,Exame_Nao_Realizado,Exame_Negativo,Exame_Positivo
1,1.0,1.0,NaN,NaN,M,1984.0,26.0,1.0,2020,0,0,0,1
2,0.0,1.0,NaN,1.0,F,1975.0,35.0,1.0,2020,1,0,0,0
4,0.0,1.0,1.0,9.0,F,2004.0,27.0,1.0,2020,0,0,0,1
5,0.0,1.0,2.0,3.0,M,1973.0,NaN,1.0,2020,1,0,0,0
6,1.0,1.0,1.0,9.0,M,1991.0,31.0,1.0,2020,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3040,1.0,9.0,1.0,9.0,M,2000.0,31.0,1.0,2023,0,1,0,0
3050,0.0,1.0,NaN,1.0,F,1969.0,31.0,1.0,2023,1,0,0,0
3052,0.0,1.0,NaN,6.0,M,1991.0,NaN,1.0,2023,0,1,0,0
3053,0.0,1.0,NaN,8.0,M,1969.0,31.0,1.0,2023,0,0,1,0


#### 2.3.4) Tratamento realizado

A categoria apresenta o tratamento que foi realizado pelo paciente no combate à doença, sendo 1=  Sim  –  Praziquantel,  2=  Sim,  Oxaminiquine,  3=  Não  ou  9=  Ignorado

In [511]:
print(esq_valid['Tratamento realizado'].value_counts())
print("Valores nulos: ", esq_valid['Tratamento realizado'].isna().sum())

1.0    4216
3.0     541
9.0     365
2.0      21
Name: Tratamento realizado, dtype: int64
Valores nulos:  0


Como não existem valores nulos, podemos apenas atribuir a cada valor numérico a string relativa ao output:

In [512]:
# Substituição dos valores numéricos por rótulos claros
esq_valid['Tratamento realizado'].replace(
    {
        1: "Sim_Praziquantel",
        2: "Sim_Oxaminiquine", 
        3: "Nao",
        9: "Ignorado"
    }, 
    inplace=True
)

In [513]:
# Criar colunas one-hot para cada categoria
df_onehot = pd.get_dummies(
    esq_valid['Tratamento realizado'], 
    prefix='Tratamento', 
    dtype=int
)

# Concatenar com o DataFrame original
esq_valid = pd.concat([esq_valid, df_onehot], axis=1)

# Remover a coluna original (opcional)
esq_valid.drop(columns=['Tratamento realizado'], inplace=True)

In [514]:
esq_valid

,Quantidade ovos encontrados,Forma clínica,Escolaridade,Sexo,Ano de nascimento,Unidade Federativa,Resultado,Ano_atual,Exame_Desconhecido,Exame_Nao_Realizado,Exame_Negativo,Exame_Positivo,Tratamento_Ignorado,Tratamento_Nao,Tratamento_Sim_Oxaminiquine,Tratamento_Sim_Praziquantel
1,1.0,NaN,NaN,M,1984.0,26.0,1.0,2020,0,0,0,1,0,0,0,1
2,0.0,NaN,1.0,F,1975.0,35.0,1.0,2020,1,0,0,0,0,0,0,1
4,0.0,1.0,9.0,F,2004.0,27.0,1.0,2020,0,0,0,1,0,0,0,1
5,0.0,2.0,3.0,M,1973.0,NaN,1.0,2020,1,0,0,0,0,0,0,1
6,1.0,1.0,9.0,M,1991.0,31.0,1.0,2020,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3040,1.0,1.0,9.0,M,2000.0,31.0,1.0,2023,0,1,0,0,1,0,0,0
3050,0.0,NaN,1.0,F,1969.0,31.0,1.0,2023,1,0,0,0,0,0,0,1
3052,0.0,NaN,6.0,M,1991.0,NaN,1.0,2023,0,1,0,0,0,0,0,1
3053,0.0,NaN,8.0,M,1969.0,31.0,1.0,2023,0,0,1,0,0,0,0,1


#### 2.3.5) Forma clínica avaliada

In [515]:
print(esq_valid['Forma clínica'].value_counts())
print("Valores nulos: ", esq_valid['Forma clínica'].isna().sum())

1.0    2677
5.0     334
3.0     263
2.0     249
4.0     186
Name: Forma clínica, dtype: int64
Valores nulos:  1434


In [516]:
# Substituindo valores nulos para "Nao_especificado"
esq_valid['Forma clínica'].fillna("Nao_especificado", inplace=True)

In [517]:
# Dicionário de mapeamento dos valores
mapeamento_forma = {
    1: "Intestinal",
    2: "Hepato_Intestinal",
    3: "Hepato_Esplenica",
    4: "Aguda",
    5: "Outra"
}

# Substituição inplace
esq_valid['Forma clínica'].replace(mapeamento_forma, inplace=True)

In [518]:
# Criando colunas one-hot
df_onehot = pd.get_dummies(
    esq_valid['Forma clínica'],
    prefix='Forma',     
    dtype=int          
)

# Concatenar com o DataFrame original
esq_valid = pd.concat([esq_valid, df_onehot], axis=1)

# Remover a coluna original (opcional)
esq_valid.drop(columns=['Forma clínica'], inplace=True)

#### 2.3.6) Escolaridade

In [519]:
print(esq_valid['Escolaridade'].value_counts())
print("Valores nulos: ", esq_valid['Escolaridade'].isna().sum())

9.0     1082
6.0      793
1.0      578
3.0      514
5.0      344
2.0      339
4.0      329
8.0      198
0.0      151
10.0     147
7.0       76
Name: Escolaridade, dtype: int64
Valores nulos:  592


Como o campo escolaridade apresenta uma cardinalidade lógica, ou seja, o 1 = 1º a 4º série incompleta e 8 = Educação superior completa, podemos considerá-lo como um atributo numérico, e não é necessário realizar o One-Hot-Encoding.

Porém, como os atributos 9 = Ignorado e 10 = Não se aplica, transformaremos-os em 0 para garantir que não sejam considerados com uma cardinalidade maior que a Educação superior completa.

In [520]:
esq_valid['Escolaridade'].fillna(0, inplace=True)
esq_valid['Escolaridade'].replace(9, 0, inplace=True)
esq_valid['Escolaridade'].replace(10, 0, inplace=True)

#### 2.3.7) Sexo

In [521]:
esq_valid['Sexo'].value_counts()

M    3066
F    2075
I       2
Name: Sexo, dtype: int64

Como temos apenas dois valores "Ignorado", podemos desconsiderá-los:

In [522]:
esq_valid['Sexo'].drop(esq_valid[esq_valid['Sexo'] == "I"].index, inplace=True)

In [523]:
esq_valid['Sexo'].replace("M", 0, inplace=True)
esq_valid['Sexo'].replace("F", 1, inplace=True)

#### 2.3.8) Ano de Nascimento

Primeiramente, precisamos extrair a idade de cada paciente conforme o ano de nascimento. 

Como foram concatenados datasets de 4 anos diferentes, usaremos a coluna "Ano_atual" a fim de calcular a idade correta de cada paciente:

In [525]:
esq_valid['Idade'] = esq_valid['Ano_atual'] - esq_valid['Ano de nascimento']

In [526]:
esq_valid.drop(columns=['Ano de nascimento', 'Ano_atual'], inplace=True)

In [527]:
esq_valid

,Quantidade ovos encontrados,Escolaridade,Sexo,Unidade Federativa,Resultado,Exame_Desconhecido,Exame_Nao_Realizado,Exame_Negativo,Exame_Positivo,Tratamento_Ignorado,Tratamento_Nao,Tratamento_Sim_Oxaminiquine,Tratamento_Sim_Praziquantel,Forma_Aguda,Forma_Hepato_Esplenica,Forma_Hepato_Intestinal,Forma_Intestinal,Forma_Nao_especificado,Forma_Outra,Idade
1,1.0,0.0,0,26.0,1.0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,36.0
2,0.0,1.0,1,35.0,1.0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,45.0
4,0.0,0.0,1,27.0,1.0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,16.0
5,0.0,3.0,0,NaN,1.0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,47.0
6,1.0,0.0,0,31.0,1.0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3040,1.0,0.0,0,31.0,1.0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,23.0
3050,0.0,1.0,1,31.0,1.0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,54.0
3052,0.0,6.0,0,NaN,1.0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,32.0
3053,0.0,8.0,0,31.0,1.0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,54.0


#### 2.3.9) Unidade Federativa

Para a utilização da unidade federativa, a escolha realizada pelo grupo foi dividir por regiões ao invés de por estados, reduzindo a complexidade total do modelo. 

Para isso, é necessário identificar cada estado e atribuí-lo à região correspondente. 